In [ ]:
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m-obb.pt")  # load a pretrained model (recommended for training)


results = model.train(
    data="data.yaml",
    epochs=50,
    batch=8,
    imgsz=256,
    name='ship_detection',
    device=0,
    single_cls=True,
    patience=10,
    # visualize=True,
    multi_scale=True,
    augment=True,
    dropout=0.1,
    # lr0=1e-4,
    # weight_decay=1e-5  # Weight Decay 줄이기
)

New https://pypi.org/project/ultralytics/8.3.5 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.4  Python-3.11.4 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=obb, mode=train, model=yolo11m-obb.pt, data=data.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=ship_detection7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pook0612 (limbw). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed 


train: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\train.cache... 357 images, 0 backgrounds, 0 corrupt: 100%|██████████| 357/357 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

       3/50      2.58G        nan        nan        nan        209        224: 100%|██████████| 45/45 [00:19<00:00,  2.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.07it/s]

                   all         37       1653          0          0          0          0



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/50      2.51G        nan        nan        nan         47        128: 100%|██████████| 45/45 [00:22<00:00,  1.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.05it/s]

                   all         37       1653          0          0          0          0
